### 1. Finray Simulation

In [ ]:
import mujoco
import mujoco_viewer
import numpy as np
import os
from lxml import etree
import lxml.etree as ET
import time
import uuid

##############################################
# Helper functions for XML attribute swapping
##############################################
def swap_par(tree, element_type, element_name, attribute_name, new_value):
    element = tree.find(f'.//{element_type}[@name="{element_name}"]')
    if element is not None:
        element.set(attribute_name, new_value)

##############################################
# Helper function to add a dynamic site to a body
##############################################
def add_dynamic_site(tree, body_name, site_name, pos):
    body_elem = tree.find(f'.//body[@name="{body_name}"]')
    if body_elem is not None:
        new_site = ET.Element("site", name=site_name, size="0.0001", pos=pos, rgba="1 0 0 1")
        body_elem.append(new_site)
    return site_name

#########################################################
# Function to generate a cantilever beam XML segment string
#########################################################
def generate_xml(ID, n):
    xml_output = []
    n_closers = n
    if isinstance(ID, str) and ID.startswith("R"):
        xml_output.append('<body>')
        xml_output.append('<joint type="free"/>')
        n_closers += 1
    xml_output.append(f'<body name="ID{ID}_b_0" pos="0 0 0" euler="0 0 0">')
    xml_output.append(f'<site name="ID{ID}_b_0_sroot" size="0.0001" pos="0 0 0"/>')
    xml_output.append(f'<geom name="ID{ID}_b_0_g" type="box" size="0.0005 0.005 0.01" pos="0 0 0.01" rgba="0.1 0.1 0.1 0.5"/>')
    xml_output.append(f'<site name="ID{ID}_b_0_s" size="0.0001" pos="0 0 0.05"/>')
    for i in range(1, n + 1):
        xml_output.append(f'<body name="ID{ID}_b_{i}" pos="0 0 0" euler="0 0 0">')
        xml_output.append(f'<joint name="ID{ID}_b_{i}_j" type="hinge" axis="0 1 0" pos="0 0 0" stiffness="0" damping="0.00001"/>')
        xml_output.append(f'<geom name="ID{ID}_b_{i}_g" type="box" size="0.0005 0.005 0.01" pos="0 0 0.01" rgba="0.1 0.1 0.1 0.5"/>')
        xml_output.append(f'<site name="ID{ID}_b_{i}_s" size="0.0001" pos="0 0 0.05"/>')
    xml_output.append(f'<site name="ID{ID}_b_{n+1}_s" size="0.0001" pos="0 0 0.05"/>')
    for i in range(n_closers + 1):
        xml_output.append('</body>')
    return '\n'.join(xml_output)

#########################################################
# Function to update geometry and dynamic parameters in XML tree
#########################################################
def reparcer(ID, geom_pars, n, tree):
    L, b, t, alpha_k = geom_pars
    l_i = L / n
    l_0 = l_i / 2
    E = 18.5e6
    I = t**3 * b / 12
    k_i = E * I / l_i
    d_i = k_i * alpha_k
    for i in range(0, n+1):
        if i == 0:
            swap_par(tree, 'geom', f'ID{ID}_b_0_g', 'size', f"{t/2} {b/2} {l_0/2}")
            swap_par(tree, 'geom', f'ID{ID}_b_0_g', 'pos', f"0 0 {l_0/2}")
            swap_par(tree, 'site', f'ID{ID}_b_0_s', 'pos', f"0 0 {l_0/2}")
        elif i == n:
            swap_par(tree, 'body', f'ID{ID}_b_{i}', 'pos', f"0 0 {l_i}")
            swap_par(tree, 'geom', f'ID{ID}_b_{i}_g', 'size', f"{t/2} {b/2} {l_0/2}")
            swap_par(tree, 'geom', f'ID{ID}_b_{i}_g', 'pos', f"0 0 {l_0/2}")
            swap_par(tree, 'site', f'ID{ID}_b_{i}_s', 'pos', f"0 0 {l_0/2}")
            swap_par(tree, 'joint', f'ID{ID}_b_{i}_j', 'stiffness', f"{k_i}")
            swap_par(tree, 'joint', f'ID{ID}_b_{i}_j', 'damping', f"{d_i}")
        else:
            pos = l_0 if i == 1 else l_i
            swap_par(tree, 'body', f'ID{ID}_b_{i}', 'pos', f"0 0 {pos}")
            swap_par(tree, 'geom', f'ID{ID}_b_{i}_g', 'size', f"{t/2} {b/2} {l_i/2}")
            swap_par(tree, 'geom', f'ID{ID}_b_{i}_g', 'pos', f"0 0 {l_i/2}")
            swap_par(tree, 'site', f'ID{ID}_b_{i}_s', 'pos', f"0 0 {l_i/2}")
            swap_par(tree, 'joint', f'ID{ID}_b_{i}_j', 'stiffness', f"{k_i}")
            swap_par(tree, 'joint', f'ID{ID}_b_{i}_j', 'damping', f"{d_i}")
            swap_par(tree, 'site', f'ID{ID}_b_{n+1}_s', 'pos', f"0 0 {l_0}")

def site_searcher(L, L_site, n):
    li = L / n
    l0 = li / 2
    segments = [l0] + [li] * (n - 1) + [l0]
    cumulative = 0.0
    for idx, seg_length in enumerate(segments):
        if cumulative + seg_length >= L_site:
            local_offset = L_site - cumulative
            return idx, local_offset
        cumulative += seg_length
    return len(segments) - 1, segments[-1]

#########################################################
# Combined function: generate a parameterized Finray model and run simulation
#########################################################
def run_finray_simulation(
    rib_angles_deg,
    thickness_ribs,
    n_springs_front,
    n_springs_back,
    n_springs_rib,
    thickness_front,
    thickness_back,
    beam_width,
    L0,
    X1,
    xf,
    zf,
    moving_cylinder_x,
    moving_cylinder_z,
    moving_cylinder_radius,
    moving_cylinder_displacement,
    vis
):
    alpha_k = 5e-3
    phi_a = np.arctan((L0 - zf) / xf)
    phi_b = -np.arctan(zf / (X1 + xf))
    beta = np.arctan(L0 / X1)
    L1 = np.sqrt(L0**2 + X1**2)

    ribs_geom = {}
    valid_rib_indices = []
    for idx, angle_deg in enumerate(rib_angles_deg):
        phi_r = np.deg2rad(angle_deg)
        if phi_r < phi_b or phi_r > phi_a:
            continue
        L_A = zf + xf * np.tan(phi_r)
        L_B = (X1 * np.tan(phi_r) + L_A) / (np.sin(beta) + np.cos(beta) * np.tan(phi_r))
        P_A_z = L_A
        P_B_x = X1 - L_B * np.sin(0.5 * np.pi - beta)
        P_B_z = L_B * np.cos(0.5 * np.pi - beta)
        L_rib = np.sqrt((0 - P_B_x)**2 + (P_A_z - P_B_z)**2)
        alpha_rib = np.arcsin((P_B_z - P_A_z) / L_rib)
        ribs_geom[idx] = {'L_A': L_A, 'L_B': L_B, 'P_A_z': P_A_z, 'L_rib': L_rib, 'alpha_rib': alpha_rib}
        valid_rib_indices.append(idx)
    valid_rib_indices = [idx for idx in valid_rib_indices if ribs_geom[idx]['P_A_z'] >= 0]

    xml_front = generate_xml("F", n_springs_front)
    xml_back = generate_xml("B", n_springs_back)
    xml_ribs = {idx: generate_xml(f"R{idx}", n_springs_rib) for idx in valid_rib_indices}

    base_xml_path = "Finray_model.xml"
    mod_xml_path = "Finray_model_mod.xml"
    
    mod_xml_path = os.path.join(tempfile.gettempdir(), f"finray_{uuid.uuid4().hex}.xml")
    
    hand_tree = ET.parse(base_xml_path)
    hand_root = hand_tree.getroot()
    eq = hand_root.find("equality")
    if eq is not None:
        eq.clear()

    worldbody = hand_root.find("worldbody")
    worldbody.append(ET.fromstring(xml_front))
    worldbody.append(ET.fromstring(xml_back))
    for xml in xml_ribs.values():
        worldbody.append(ET.fromstring(xml))
    focal_site = ET.Element("site", name="focal_site", pos=f"{-xf} 0 {zf}", size="0.001", rgba="0 1 0 0.1", type="sphere")
    worldbody.append(focal_site)

    for idx in valid_rib_indices:
        start = np.array([-xf, 0, zf])
        end = np.array([0, 0, ribs_geom[idx]['P_A_z']])
        vec = end - start
        length = np.linalg.norm(vec)
        midpoint = start + vec/2
        angle = np.arccos(vec[2] / length)
        q0, q2 = np.cos(angle/2), np.sin(angle/2)
        geom = ET.Element("geom", name=f"focal_line_{idx}", type="cylinder",
                          pos=f"{midpoint[0]} {midpoint[1]} {midpoint[2]}",
                          quat=f"{q0} 0 {q2} 0",
                          size=f"0.0005 {length/2}", rgba="0 1 0 0.1", contype="0", conaffinity="0")
        worldbody.append(geom)

    hand_tree.write(mod_xml_path, pretty_print=True)
    tree = etree.parse(mod_xml_path)
    root = tree.getroot()
    worldbody = root.find("worldbody")

    # Update beam parameters
    reparcer("F", [L0, beam_width, thickness_front, alpha_k], n_springs_front, tree)
    reparcer("B", [L1, beam_width, thickness_back, alpha_k], n_springs_back, tree)
    for idx in valid_rib_indices:
        rib_t = thickness_ribs[idx] if thickness_ribs and idx < len(thickness_ribs) else thickness_rib
        reparcer(f"R{idx}", [ribs_geom[idx]['L_rib'], beam_width, rib_t, alpha_k], n_springs_rib, tree)

    # Position back beam
    swap_par(tree, 'body', 'IDB_b_0', 'euler', f"0 {-np.rad2deg(0.5*np.pi - beta)} 0")
    swap_par(tree, 'body', 'IDB_b_0', 'pos', f"{X1} 0 0")
    # Position ribs
    for idx in valid_rib_indices:
        swap_par(tree, 'body', f'IDR{idx}_b_0', 'euler', f"0 {90 - np.rad2deg(ribs_geom[idx]['alpha_rib'])} 0")
        swap_par(tree, 'body', f'IDR{idx}_b_0', 'pos', f"0 0 {ribs_geom[idx]['P_A_z']}")

    # Connections
    conn_sites = {}
    for idx in valid_rib_indices:
        i_f, l_f = site_searcher(L0, ribs_geom[idx]['L_A'], n_springs_front)
        i_b, l_b = site_searcher(L1, ribs_geom[idx]['L_B'], n_springs_back)
        sf, sb = f"IDF_b_{i_f}_conn_{idx}", f"IDB_b_{i_b}_conn_{idx}"
        add_dynamic_site(tree, f"IDF_b_{i_f}", sf, f"0 0 {l_f}")
        add_dynamic_site(tree, f"IDB_b_{i_b}", sb, f"0 0 {l_b}")
        conn_sites[idx] = (sf, sb)

    eq = tree.find("equality")
    if eq is None:
        eq = ET.SubElement(root, "equality")
    eq.append(ET.Element("weld", name="connect_fin", site1=f"IDF_b_{n_springs_front+1}_s", site2=f"IDB_b_{n_springs_back+1}_s"))
    for idx in valid_rib_indices:
        sf, sb = conn_sites[idx]
        eq.append(ET.Element("weld", name=f"connect_rib_A{idx}", site1=sf, site2=f"IDR{idx}_b_0_sroot"))
        eq.append(ET.Element("weld", name=f"connect_rib_B{idx}", site1=sb, site2=f"IDR{idx}_b_{n_springs_rib+1}_s"))

    # Moving cylinder body, joint, geom and force site
    moving_body = ET.Element("body", name="moving_cylinder", pos=f"{-moving_cylinder_x} 0 {moving_cylinder_z}", euler="90 0 0")
    moving_joint = ET.Element("joint", name="slider_joint", type="slide", axis="1 0 0", limited="true", range="-0.5 0.5", damping="1")
    
    moving_body.insert(0, moving_joint)
    moving_geom = ET.Element("geom", name="moving_cylinder_geom", type="cylinder",
                             pos="0 0 0", size=f"{moving_cylinder_radius} 0.1",
                             rgba="0.8 0.2 0.2 0.4", euler="0 0 90",
                             condim="4", contype="1", conaffinity="15",
                             friction="0.9 0.2 0.2", solref="0.001 2")
    moving_body.append(moving_geom)
    # Add site for force-based actuator
    moving_site = ET.Element("site", name="cylinder_site", pos="0 0 0", size="0.001", rgba="0 0 1 1")
    moving_body.append(moving_site)
    worldbody.append(moving_body)

    # Define general (force) actuator
    
    #actuator_elem = root.find("actuator") or ET.SubElement(root, "actuator")
    #actuator_elem.clear()
    #general_elem = ET.Element("general", name="applied_force", site="cylinder_site", gear="1 0 0")
    #actuator_elem.append(general_elem)
    
    actuator_elem = root.find("actuator")
    if actuator_elem is None:
        actuator_elem = ET.Element("actuator")
        root.append(actuator_elem)
    position_elem = ET.Element("position", name="cylinder_actuator", joint="slider_joint", kp="20000", dampratio="1", ctrlrange="-0.5 0.5")
    actuator_elem.append(position_elem)

    tree.write(mod_xml_path, pretty_print=True, xml_declaration=True, encoding='UTF-8')

    #########################################################
    # Run simulation
    #########################################################
    model = mujoco.MjModel.from_xml_path(mod_xml_path)
    data = mujoco.MjData(model)
    viewer = mujoco_viewer.MujocoViewer(model, data, title=mod_xml_path, width=854, height=480)
    
    cam_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_CAMERA, "side view")
    viewer.cam.type       = mujoco.mjtCamera.mjCAMERA_FIXED   # тип — fixed camera :contentReference[oaicite:3]{index=3}
    viewer.cam.fixedcamid = cam_id 

    sim_time = 8
    timestep = 0.001
    n_sites = n_springs_front + 2
    pos_init = np.zeros((n_sites, 2))
    pos_deform = np.zeros((n_sites, 2))
    STEP_NUM = int(sim_time / timestep)
    
    contact_points_X = []
    contact_points_Z = []
    
    contact_forces_X = []
    contact_forces_Z = []

    act_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, 'cylinder_actuator')

    for i in range(STEP_NUM):
        if not viewer.is_alive:
            break
        data.ctrl[act_id] = moving_cylinder_displacement
        if i == 1:
            for j in range(n_sites):
                p = data.site_xpos[j+1]
                pos_init[j] = [p[0], p[2]]
        if i == STEP_NUM-1:
            for j in range(n_sites):
                p = data.site_xpos[j+1]
                pos_deform[j] = [p[0], p[2]]
        mujoco.mj_step(model, data)
        
        # Сбор данных только в последнем кадре
        if i == STEP_NUM - 1:
            for contact_idx in range(data.ncon):
                contact = data.contact[contact_idx]
                
                # Extract contact points
                pos = np.array(contact.pos)
                contact_points_X.append(pos[0])
                contact_points_Z.append(pos[2])
                
                # Extract contact forces
                ft = np.zeros(6)
                mujoco.mj_contactForce(model, data, contact_idx, ft)
                contact_xmat = np.array(contact.frame).reshape(3,3).T
                world_f = contact_xmat @ ft[:3]
                contact_forces_X.append(world_f[0])
                contact_forces_Z.append(world_f[2])
      
        if vis:
            viewer.render()

    viewer.close()
    os.remove(mod_xml_path)
    
    dx = np.diff(contact_points_X)
    dz = np.diff(contact_points_Z)
    segment_lengths = np.sqrt(dx**2 + dz**2)
    con_len = np.sum(segment_lengths) # total curve length    
    con_S = con_len * beam_width # contact area
    
    front_joint_ids = [mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, f"IDF_b_{i}_j")
                       for i in range(1, n_springs_front+1)]
    back_joint_ids  = [mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, f"IDB_b_{i}_j")
                       for i in range(1, n_springs_back+1)]

    # Извлекаем текущие углы (qpos) и жесткости (из модели)
    E_front = 0.0
    for jid in front_joint_ids:
        theta = data.qpos[jid]                  # угол сочленения
        k     = model.jnt_stiffness[jid]        # жесткость
        E_front += 0.5 * k * theta**2
    
    E_back = 0.0
    for jid in back_joint_ids:
        theta = data.qpos[jid]
        k     = model.jnt_stiffness[jid]
        E_back += 0.5 * k * theta**2

    # и возвращаем их вместе с остальными данными
    return pos_init, pos_deform, contact_forces_X, contact_forces_Z, \
           contact_points_X, contact_points_Z, con_S, E_front, E_back

### 2. Parametric Synthesis

In [ ]:
import mujoco
import mujoco_viewer
import numpy as np
import time
from scipy.optimize import differential_evolution
from scipy.optimize import NonlinearConstraint
import tempfile

n_springs_front = 20
n_springs_back  = 20
n_springs_rib   = 12
L0 = 70e-3
X1 = 35e-3
beam_width = 35e-3

moving_cylinder_radius = 15e-3
moving_cylinder_displacement = 15e-3

moving_cylinder_z = 35e-3

def aggregate_metrics(winding_number, area_factor, sum_fx, sum_fz,
                      bounds):

    # Распакуем границы

    fx_min, fx_max = bounds['fx']
    fz_min, fz_max = bounds['fz']
    
    # Нормировка для метрик, которые надо *максимизировать*
    def norm_max(x, x_min, x_max):
        return (x - x_min) / (x_max - x_min)
    
    # Нормировка для метрик, которые надо *минимизировать*
    def norm_min(x, x_min, x_max):
        return (x_max - x) / (x_max - x_min)
    
    # Применяем нормировку
    nfx = norm_max(sum_fx,     fx_min, fx_max)
    nfz = norm_min(sum_fz,     fz_min, fz_max)

    print(f"mormalized sum FX: {nfx}")
    print(f"mormalized sum FZ: {nfz}")
    
    total_score = - winding_number - area_factor - nfx + nfz
    return total_score

def get_best_finray(X):
    th_front, th_back, xf, zf = X[:4]
    n_rib = (len(X) - 4) // 2
    phi_ribs = X[4 : 4 + n_rib].tolist()
    th_ribs  = X[4 + n_rib :].tolist()
    
    # Run simulation
    pos_init, pos_deform, con_Fx, con_Fz, con_Px, con_Pz, con_S, E_front, E_back = run_finray_simulation(
        rib_angles_deg = phi_ribs,
        thickness_ribs = th_ribs,
        n_springs_front = n_springs_front,
        n_springs_back  = n_springs_back,
        n_springs_rib   = n_springs_rib,
        thickness_front = th_front,
        thickness_back  = th_back,
        beam_width      = beam_width,
        L0              = L0,
        X1              = X1,
        xf              = xf,
        zf              = zf,
        moving_cylinder_x           = moving_cylinder_radius + th_front / 2,
        moving_cylinder_z           = moving_cylinder_z,
        moving_cylinder_radius      = moving_cylinder_radius,
        moving_cylinder_displacement = moving_cylinder_displacement,
        vis             = True
    )
    
    
    fx_arr = np.sum(np.array(con_Fx))
    fz_arr = np.sum(np.array(con_Fz))
    S_arr = np.array(con_S)
    S_max = L0 * beam_width
    area_factor = S_arr/S_max
    
    metric_bounds = {
    'fx':      (0, 50.0),
    'fz':      (-10.0, 10.0),
    }

    
    angles = []
    # Winding number
    angles = np.arctan2(pos_deform[:,1], pos_deform[:,0])
    angles_unwrapped = np.unwrap(angles)
    delta_angle = angles_unwrapped[-1] - angles_unwrapped[0]
    winding_number = delta_angle / (2 * np.pi)

    print(f"Winding number: {winding_number}")
    print(f"Area factor: {S_arr/S_max}")
    print(f"Sum(FX): {fx_arr}")
    print(f"Sum(FZ): {fz_arr}")
    
    
    reward = aggregate_metrics(
        winding_number, area_factor, fx_arr, fz_arr,
        bounds=metric_bounds
    )
    
    print(f"reward = {reward}")
    
    return reward

def build_optimization(n_rib):
    # Initial guesses
    th_front0 = 2e-3
    th_back0  = 2e-3
    xf0       = 10e-3
    zf0       = 35e-3
    
    phi_a = np.rad2deg(np.arctan((L0 - zf0) / xf0))
    phi_b = np.rad2deg(-np.arctan(zf0 / (X1 + xf0)))

    if n_rib > 0:
        phi0 = np.linspace(phi_b, phi_a, n_rib)
    else:
        phi0 = np.array([])
    
    th_ribs0  = np.ones(n_rib) * 2e-3
    X_init    = np.hstack([th_front0, th_back0, xf0, zf0, phi0, th_ribs0])

    # Bounds
    bnds = []
    bnds += [(2e-3, 3e-3), (3e-3, 5e-3)]      # thickness_front, thickness_back
    bnds += [(1e-3, 2000e-3), (-1000e-3, 1000e-3)]         # focal coords xf, zf
    bnds += [(-60, 60)] * n_rib  # rib angles phi
    bnds += [(2e-3, 3e-3)] * n_rib               # rib thickness

    return X_init, bnds

def angle_constraints(X, n_rib):
    # X = [th_front, th_back, xf, zf, phi1..phi_n, th_rib1..th_rib_n]
    xf, zf = X[2], X[3]
    phis = X[4:4+n_rib]
    # Здесь ваша логика расчёта φ_b, φ_a из xf,zf:
    phi_a = np.rad2deg(np.arctan((L0 - zf) / xf))
    phi_b = np.rad2deg(-np.arctan(zf / (X1 + xf)))
    
    # Для каждого ребра два неравенства:
    lower = phis - phi_b             # phi_i ≥ phi_b
    upper = phi_a - phis             # phi_i ≤ phi_a
    return np.hstack([lower, upper])

def phi_constr_cone(xf, t, phi):
    if xf <= 0:
        return 0.0, 0.0
    # Расстояние от фокуса до ребра «вдоль» оси
    a = xf / np.cos(phi)
    # Горизонтальная «полуширина» ребра
    b = (t / 2) / np.cos(phi)
    # Длины вспомогательных сторон треугольников
    c = np.sqrt(a**2 + b**2 - 2*a*b*np.cos(np.pi/2 - phi))
    d = np.sqrt(a**2 + b**2 - 2*a*b*np.cos(np.pi/2 + phi))
    # Угловые допуски по формулам косинусов
    delta_minus = np.arccos((a**2 + c**2 - b**2) / (2*a*c))
    delta_plus  = np.arccos((a**2 + d**2 - b**2) / (2*a*d))
    return delta_minus, delta_plus

def nonintersect_constraints(X, n_rib):
    if n_rib < 2:
        return np.array([], dtype=float)

    xf, zf = X[2], X[3]
    phis    = np.deg2rad(X[4:4+n_rib])
    ths     = X[4+n_rib:4+2*n_rib]
    # 1) сортируем
    order = np.argsort(phis)
    phs   = phis[order]
    ths   = ths[order]

    # 2) расчёт δ
    delta_minus = np.zeros(n_rib)
    delta_plus  = np.zeros(n_rib)
    for i in range(n_rib):
        dm, dp = phi_constr_cone(xf, ths[i], phs[i])
        delta_minus[i], delta_plus[i] = dm, dp

    # 3) gᵢ ≥ 0: если все хорошо, возвращаем строго положительные eps
    g = np.empty(n_rib-1)
    for i in range(n_rib-1):
        g[i] = (phs[i+1] - delta_minus[i+1]) - (phs[i] + delta_plus[i])
        # небольшая подтяжка в плюсовую зону, чтобы не было нулей
        g[i] = g[i] if g[i] >= 0 else g[i]
    return g

history = []

def de_callback(xk, convergence):
    fval = get_best_finray(xk)
    history.append((xk.copy(), fval))
    # возвращаем False, чтобы DE не прерывался досрочно
    return False

n_rib = 2

X_init, bnds = build_optimization(n_rib)
print(f"Search space of {len(X_init)} parameters")

nlc_angle = NonlinearConstraint(
    fun=lambda X: angle_constraints(X, n_rib),
    lb=0, ub=np.inf
)

nlc_nonint = NonlinearConstraint(
    fun=lambda X: nonintersect_constraints(X, n_rib),
    lb=0.0,  # g_i >= 0
    ub=np.inf,
    keep_feasible=True
)

constraints = (nlc_angle, nlc_nonint)

from multiprocessing import Pool

res = differential_evolution(
    get_best_finray,
    bounds = bnds,
    constraints = constraints,
    maxiter = 10,
    popsize = len(X_init) * 10,
    mutation = (0.5,1),
    recombination = 0.7,
    updating = 'immediate',
    disp = True,
    polish = False
) 

print('Optimal result:', res)

In [ ]:
optimal_X = res.x

opt = res.x
th_front_opt   = opt[0]
th_back_opt    = opt[1]
xf_opt         = opt[2]
zf_opt         = opt[3]
phi_ribs_opt   = opt[4 : 4 + n_rib]
th_ribs_opt    = opt[4 + n_rib : 4 + 2*n_rib]


th_ribs_opt = list(th_ribs_opt) if th_ribs_opt is not None else []

start_time = time.time()

pos_init, pos_deform, con_Fx, con_Fz, con_Px, con_Pz, con_S, E_front, E_back = run_finray_simulation(
    rib_angles_deg = phi_ribs_opt,
    thickness_ribs = th_ribs_opt,
    n_springs_front = n_springs_front,
    n_springs_back = n_springs_back,
    n_springs_rib = n_springs_rib,
    thickness_front = th_front_opt,
    thickness_back = th_back_opt,
    beam_width = beam_width,
    L0 = L0,
    X1 = X1,
    xf = xf_opt,
    zf = zf_opt,
    moving_cylinder_x = moving_cylinder_radius + th_front_opt/2,
    moving_cylinder_z = moving_cylinder_z + 5e-3,
    moving_cylinder_radius = moving_cylinder_radius,
    moving_cylinder_displacement = moving_cylinder_displacement,
    vis = True
)



pos_deform = np.array(pos_deform)

# Извлекаем первый столбец и находим максимум
max_x = np.max(pos_deform[:, 0])


print(f"Inflection: {max_x*1e3} mm")

print(f"Sum of contact forces X:, {sum(con_Fx)}, N")
print(f"Sum of contact forces Z:, {sum(con_Fz)}, N")
print(f"Contact area:, {con_S*1e6}, mm2")

print(f"Simulation time: {time.time() - start_time:.4f} s")    
